In [78]:
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [79]:
data_dir = ('./data/')
class_names = ["apple", "bat", "cat", "circle", "clock", "cloud",
               "crown", "diamond", "dog", "donut", "fish",
                "hot_dog", "lightning", "mountain", "river", "skull",
                "smiley_face", "square", "star", "sun", "t-shirt", "tree"]

In [80]:
X = []
y = []

In [81]:
for label, class_name in enumerate(class_names):
    # Create the path to the .npy file (e.g., './data/apple.npy')
    file_path = os.path.join(data_dir, f"{class_name}.npy")

    # Load the .npy file → shape: (2000, 28, 28)
    data = np.load(file_path)

    X.append(data)

    # Add labels to y → [label, label, label, ..., label] (length = number of samples)
    y.append(np.full((data.shape[0]), label))


In [82]:
# Stack all image arrays vertically → final shape: (total_samples, 28, 28)
X = np.vstack(X)

# Stack all label arrays horizontally → final shape: (total_samples,)
y = np.hstack(y)

In [83]:
X = StandardScaler().fit_transform(X)

### Max Normalization

In [84]:
# # Normalize pixel values to range 0–1 (from 0–255)
# X = X.astype('float32') / 255.0
# X.shape

In [85]:
# Flatten each image from 28x28 → 784 for dense layers
# X = X.reshape(X.shape[0], -1)  # shape becomes (total_samples, 784)

In [86]:
X_train, X_, y_train, y_ = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
# `stratify=y` ensures equal class distribution in both training and validation sets.

X_cv, X_test, y_cv, y_test = train_test_split(X_,y_, test_size=0.5, random_state=42, stratify=y_)

In [87]:
model = Sequential([
    Input(shape=(784,), name="input_vector"),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.4),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.2),
    Dense(len(class_names), activation='linear')
])

In [88]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [89]:
# Train
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

In [90]:
model.fit(X_train, y_train, validation_data=(X_cv, y_cv), epochs=50, callbacks=callbacks)

Epoch 1/50
16500/16500 ━━━━━━━━━━━━━━━━━━━━ 97s 6ms/step - accuracy: 0.6532 - loss: 1.8109 - val_accuracy: 0.7698 - val_loss: 1.2714 - learning_rate: 0.0010
Epoch 2/50
16500/16500 ━━━━━━━━━━━━━━━━━━━━ 93s 6ms/step - accuracy: 0.7232 - loss: 1.4337 - val_accuracy: 0.7818 - val_loss: 1.2279 - learning_rate: 0.0010
Epoch 3/50
16500/16500 ━━━━━━━━━━━━━━━━━━━━ 93s 6ms/step - accuracy: 0.7303 - loss: 1.4074 - val_accuracy: 0.7861 - val_loss: 1.1972 - learning_rate: 0.0010
Epoch 4/50
16500/16500 ━━━━━━━━━━━━━━━━━━━━ 95s 6ms/step - accuracy: 0.7337 - loss: 1.3881 - val_accuracy: 0.7887 - val_loss: 1.1776 - learning_rate: 0.0010
Epoch 5/50
16500/16500 ━━━━━━━━━━━━━━━━━━━━ 90s 5ms/step - accuracy: 0.7355 - loss: 1.3753 - val_accuracy: 0.7871 - val_loss: 1.1821 - learning_rate: 0.0010
Epoch 6/50
16500/16500 ━━━━━━━━━━━━━━━━━━━━ 90s 5ms/step - accuracy: 0.7360 - loss: 1.3657 - val_accuracy: 0.7865 - val_loss: 1.1756 - learning_rate: 0.0010
Epoch 7/50
16500/16500 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step -

In [91]:
y_cv_pred_probs = model.predict(X_cv)   #predict probability for each label
y_cv_pred = np.argmax(y_cv_pred_probs, axis=1)  # selects the label with maximum probability

2063/2063 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [92]:
print(accuracy_score(y_cv,y_cv_pred))

0.865560606060606


In [93]:
y_test_pred_probs = model.predict(X_test) 
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
print(accuracy_score(y_test,y_test_pred))

2063/2063 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
0.8652878787878788


In [94]:
model.save("doodle_model.keras")